# Dataset Creator

Before training an ANN on a dataset, it is needed to collect training data. This notebook has the scope of presenting an useful tool for creating a suitable dataset.

In [1]:
!pip install py_vollib

## Pricing Support Function

Support methods for pricing functions. Specifically, here used FFT technique is used.

In [2]:
import numpy as np

def FFTMethod_CM(f, methodParams, moneyness):

    x1=methodParams['x1']
    xN=methodParams['xN']
    dx=methodParams['dx']
    z1=methodParams['z1']
    zN=methodParams['zN']
    dz=methodParams['dz']
    M=methodParams['M']
    
    N=2**M;
    x=np.arange(x1,xN+dx,dx)
    z=np.arange(z1,zN+dz,dz)
    vect=np.linspace(0,N-1,N)
    
    fj=np.exp(-1j*z1*vect*dx)*f(x,0) # define fj
    FFT=np.fft.fft(fj)
    Integrals=np.real(dx*np.exp(-1j*x1*z)*FFT)

    I = np.interp(np.exp(-moneyness), np.exp(z), Integrals)
    
    return I




def setNumericalParams(M,dz):
    
    N=2**M
    z1=-dz*(N-1)/2
    dx=2*np.pi/(N*dz)
    x1=-dx*(N-1)/2
    zN=z1+(N-1)*dz
    xN=x1+(N-1)*dx
    
    methodParams={'x1':x1, 'xN':xN, 'dx':dx, 'z1':z1, 'zN':zN, 'dz':dz, 'M':M}
    
    return methodParams

## Pricing Function

In the following cells:

- *Monte Carlo pricing*: not the main pricing technique, it is presented as a support method
- *Carr-Madan pricing*: the used pricing method (plus some auxilliary functions)
- *Implied Volatility*: implied volatility finder functions

In [2]:
def priceVGSSD_MC(moneyness, TTM, sigma, nu, theta, gamma, N_MC = int(1e7)):

    g = np.random.randn(N_MC)
    G = np.random.gamma(1/nu, nu, N_MC)

    CF_norm = (1 - theta*nu*TTM**(gamma) - 0.5*sigma**2*nu*TTM**(2*gamma) )**(-1/nu)
    Y_t = TTM ** gamma * (theta * G + sigma * np.sqrt(G) * g)
    f_t = -np.log(CF_norm) + Y_t
    Ft = np.exp(f_t)
    payoff = (Ft - np.exp(-moneyness)).clip(0)
    prices = np.mean(payoff)

    return prices


def priceNIGSSD_MC(moneyness, TTM, sigma, nu, theta, gamma, N_MC = int(1e7)):

    g = np.random.randn(N_MC)
    G = np.random.wald(1/nu, 1, N_MC)

    CF_norm = np.exp(-sigma * (np.sqrt(nu**2/sigma**2 + theta**2/sigma**4-(theta/sigma**2 + TTM**gamma)**2) - nu/sigma))
    Y_t = TTM ** gamma * (theta * G + sigma * np.sqrt(G) * g)
    f_t = -np.log(CF_norm) + Y_t
    Ft = np.exp(f_t)
    
    payoff = (Ft - np.exp(-moneyness)).clip(0)
    prices = np.mean(payoff)

    return prices

def AnalyticVG(sigma,nu,theta,gamma,T):
    return (1 - theta*nu*T[0]**(gamma) - 0.5*sigma**2*nu*T[0]**(2*gamma)) > 0 and \
            (1 - theta*nu*T[-1]**(gamma) - 0.5*sigma**2*nu*T[-1]**(2*gamma))> 0

    
def AnalyticNIG(sigma,nu,theta,gamma,T):
    return (nu**2/sigma**2 + theta**2/sigma**4-(theta/sigma**2 + T[0]**gamma)**2) > 0 and \
            (nu**2/sigma**2 + theta**2/sigma**4-(theta/sigma**2 + T[-1]**gamma)**2)> 0

In [4]:
def CallPrices_CM(phiY, r, moneyness, TTM, methodParams):
    
    phi = lambda y: np.exp(-1j*y*np.log(phiY(-1j)))*phiY(y)
    
    fun = lambda y: np.exp(1j*r*y*TTM)*(phi(y-1j)-1)/(1j*y*(1j*y+1));
    
    f = lambda y,z: fun(y)*np.exp(-1j*z*y) # function to integrate
    I = FFTMethod_CM(f, methodParams, moneyness) 

    prices = I/(2*np.pi) + (1-np.exp(-moneyness-r*TTM)).clip(0)
        
    return prices

def PriceVG(moneyness, TTM, sigma, nu, theta, gamma):
    
    phiY = lambda y: (1 - 1j*y*theta*nu*TTM**gamma + 0.5*sigma**2*nu*y**2*TTM**(2*gamma))**(-1/nu) # VGSSD characteristic function
    price = CallPrices_CM(phiY, 0, moneyness, TTM, methodParams)
    
    if price>0 and price < 1e2:
        return price
    elif abs(price)<1e-4 :
        return 0
    else:
        print("price_CM = "+str(price)+" TTM ="+str(TTM)+ " moneyness ="+str(moneyness)+" params: ("+str(sigma)+", "+str(nu)+", "+str(theta)+", "+str(gamma)+")")
        price = priceVGSSD_MC(moneyness, TTM, sigma, nu, theta, gamma)
    
    if price>0 and price < 1e2:
        return price
    else:
        print("price_MC = "+str(price)+" TTM ="+str(TTM)+ " moneyness ="+str(moneyness)+" params: ("+str(sigma)+", "+str(nu)+", "+str(theta)+", "+str(gamma)+")")
    return 0
    
def PriceNIG(moneyness, TTM, sigma, nu, theta, gamma):
    
    phiY = lambda y: np.exp(-sigma*(np.sqrt(nu**2/sigma**2 + theta**2/sigma**4-(theta/sigma**2+1j*y*TTM**gamma)**2)-nu/sigma))  # NIGSSD characteristic function
    price = CallPrices_CM(phiY, 0, moneyness, TTM, methodParams)
    
    if price>0 and price < 1e2:
        return price
    elif abs(price)<1e-4:
        return 0
    else:
        print("price_CM = "+str(price)+" TTM ="+str(TTM)+ " moneyness ="+str(moneyness)+" params: ("+str(sigma)+", "+str(nu)+", "+str(theta)+", "+str(gamma)+")")
        price = priceNIGSSD_MC(moneyness, TTM, sigma, nu, theta, gamma)
    
    if price>0 and price < 1e2:
        return price
    else:
        print("price_MC = "+str(price)+" TTM ="+str(TTM)+ " moneyness ="+str(moneyness)+" params: ("+str(sigma)+", "+str(nu)+", "+str(theta)+", "+str(gamma)+")")
    return 0
    
    
def volaFinder(model,moneyness, TTM, sigma, nu, theta, gamma):
    if model == "VG":
        return impVol(PriceVG(moneyness, TTM, sigma, nu, theta, gamma), np.exp(-moneyness), TTM)
    else:
        return impVol(PriceNIG(moneyness, TTM, sigma, nu, theta, gamma), np.exp(-moneyness), TTM)
    
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv
def impVol(price,K,TTM):
    try:
        vol = iv(price,1,K,TTM,0,'c')
    except:
        vol = 0
    return vol

In [5]:
from scipy.stats import norm
from scipy.special import ndtr

def bs_call(K, T, vol):
    d1 = (np.log(1/K) + (0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return ndtr(d1) - K * ndtr(d2)

def bs_vega(K, T, sigma):
    d1 = (np.log(1 / K) + (0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return norm._pdf(d1) * np.sqrt(T)

def find_vol(target_value, K, T, *args):
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-4
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_call(K, T, sigma)
        vega = bs_vega(K, T, sigma)
        diff = target_value - price  # our root
        if (abs(diff) < PRECISION):
            return sigma

        sigma = sigma + diff/vega # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

In [6]:
F0 = 1
B = 1

strikes = np.array([0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5])
maturities = np.array([0.1, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.0])

maturities_dim = len(maturities)
strikes_dim = len(strikes)

lb = [0.3,0.2,-2,0.3]
ub = [1,1,-0.2,0.6]

param_steps = 18
param_vec = [np.linspace(lb[i],ub[i],param_steps) for i in range(4)]

M = 15
dz = 0.001
methodParams = setNumericalParams(M, dz)
moneyness = np.log(F0 / strikes)
moneyness[moneyness == np.inf] = 0

## Creating Dataset

Dataset creation is composed of three phases:
1. Pricing and implied volatility surface for each combination of parameters
2. Cleaning from parameters combination resulting in out-of-scope results
3. Saving data

In [7]:
num_scenarios = param_steps**4

idx = 0

idxes = [0]*num_scenarios; idx_time=0
training_vec = np.zeros((num_scenarios,maturities_dim*strikes_dim+4))

for sigma in param_vec[0]:
    for nu in param_vec[1]:
        for theta in param_vec[2]:
            for gamma in param_vec[3]:
                
                idx_time+=1
                disp=str(idx_time)+"/"+str(num_scenarios)
                print (disp,end="\r")
                
                if AnalyticVG(sigma,nu,theta,gamma,maturities):
                    
                    temp_vec = np.array([sigma,nu,theta,gamma] + [volaFinder("VG",moneyness[k], maturities[t], sigma, nu, theta, gamma)
                                         for t in range(maturities_dim) for k in range(strikes_dim)])
                    
                    training_vec[idx]=temp_vec
                    idxes[idx_time-1]=1
                    idx+=1


In [8]:
dat2=[]

def keepTheValue(el):
    for i in el:
        if i == 0:
            return False
    return True

for i in range(len(training_vec)):
    if(keepTheValue(training_vec[i])):
        dat2.append(training_vec[i])

In [1]:
import gzip

f = gzip.GzipFile("VGSSDTrainSet.txt.gz", "w")
np.save(file=f, arr=dat2)
f.close()

NameError: name 'np' is not defined

## Extras

1. FFT pricing using Lewis formula - in practice I used Carr-Madan.
2. Implemented volatility surface finder - not as efficient as the one used.

In [10]:
"""
priceFFT block

F0 and B are kept in the formulation for sake of keeping the function general. For the use of this Notebook they are
both set to 1.
"""

def FFTMethod(f, methodParams, moneyness):
      
    x1=methodParams['x1']
    xN=methodParams['xN']
    dx=methodParams['dx']
    z1=methodParams['z1']
    zN=methodParams['zN']
    dz=methodParams['dz']
    M=methodParams['M']
    
    N=2**M;
    x=np.arange(x1,xN+dx,dx)
    z=np.arange(z1,zN+dz,dz)
    vect=np.linspace(0,N-1,N)
    
    fj=np.exp(-1j*z1*vect*dx)*f(x,0) # define fj
    FFT=np.fft.fft(fj)
    Integrals=np.real(dx*np.exp(-1j*x1*z)*FFT)

    I = np.interp(moneyness, z, Integrals)
    
    return I




import numpy as np

def PutPricesVGSSDFFT(F0, B, moneyness, TTM, sigma, nu, theta, gamma, methodParams):
    
    if theta - (2 - nu * sigma ** 2 * TTM ** (2 * gamma)) / (2 * nu * TTM ** gamma)>0:
        return 1e10
    
    phiY = lambda y: (1 - 1j*y*theta*nu*TTM**gamma + 0.5*sigma**2*nu*y**2*TTM**(2*gamma))**(-1/nu) # VGSSD characteristic function

    phi = lambda y: np.exp(-1j*y*np.log(phiY(-1j)))*phiY(y)  
    f=lambda y,z: phi(-y-1j/2)*np.exp(-1j*z*y)/(y**2+0.25) # function to integrate

    I = FFTMethod(f, methodParams, moneyness) 
    
    prices = (np.exp(-moneyness)-np.exp(-moneyness/2)/(2*np.pi)*I)*F0*B
        
    return prices


def CallPricesVGSSDFFT(F0, B, moneyness, TTM, sigma, nu, theta, gamma, methodParams):
    
    if theta - (2 - nu * sigma ** 2 * TTM ** (2 * gamma)) / (2 * nu * TTM ** gamma)>0:
        return 1e10
    
    phiY = lambda y: (1 - 1j*y*theta*nu*TTM**gamma + 0.5*sigma**2*nu*y**2*TTM**(2*gamma))**(-1/nu) # VGSSD characteristic function

    phi = lambda y: np.exp(-1j*y*np.log(phiY(-1j)))*phiY(y)
    
    f=lambda y,z: phi(-y-1j/2)*np.exp(-1j*z*y)/(y**2+0.25) # function to integrate
    I = FFTMethod(f, methodParams, moneyness) 
        
    prices = (1-np.exp(-moneyness/2)/(2*np.pi)*I)*F0*B
        
    return prices


def CallPricesNIGSSDFFT(F0, B, moneyness, TTM, sigma, nu, theta, gamma, methodParams):

    if theta - (nu - TTM** (2 * gamma) * sigma)/ (2*TTM**gamma) > 0:
        return 1e10
    
    phiY = lambda y: np.exp(-sigma*(np.sqrt(nu**2/sigma**2 + theta**2/sigma**4-(theta/sigma**2+1j*y*TTM**gamma)**2)-nu/sigma))  # NIGSSD characteristic function

    phi = lambda y: np.exp(-1j*y*np.log(phiY(-1j)))*phiY(y)  
    
    f = lambda y, z: phi(-y-1j/2)*np.exp(-1j*z*y)/(y**2+0.25)  # function to integrate
    I = FFTMethod(f, methodParams, moneyness) 
    
    prices = (1-np.exp(-moneyness/2)/(2*np.pi)*I)*F0*B
        
    return prices


def PutPricesNIGSSDFFT(F0, B, moneyness, TTM, sigma, nu, theta, gamma, methodParams):
    
    if theta - (nu - TTM** (2 * gamma) * sigma)/ (2*TTM**gamma) > 0:
        return 1e10
    
    phiY = lambda y: np.exp(-sigma*(np.sqrt(nu**2/sigma**2 + theta**2/sigma**4-(theta/sigma**2+1j*y*TTM**gamma)**2)-nu/sigma)) # NIGSSD characteristic function

    phi = lambda y: np.exp(-1j*y*np.log(phiY(-1j)))*phiY(y)  
    
    f = lambda y,z: phi(-y-1j/2)*np.exp(-1j*z*y)/(y**2+0.25)  # function to integrate
    I = FFTMethod(f, methodParams, moneyness) 
    
    prices = (np.exp(-moneyness)-np.exp(-moneyness/2)/(2*np.pi)*I)*F0*B
        
    return prices

In [11]:
import scipy.stats as si

def newton_vol_call(S, K, T, C, r, sigma):
    
    d1 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    fx = S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0) - C
    
    vega = (1 / np.sqrt(2 * np.pi)) * S * np.sqrt(T) * np.exp(-(si.norm.cdf(d1, 0.0, 1.0) ** 2) * 0.5)
    
    tolerance = 0.000001
    x0 = sigma
    xnew  = x0
    xold = x0 - 1
        
    while abs(xnew - xold) > tolerance:
    
        xold = xnew
        xnew = (xnew - fx - C) / vega
        
        return abs(xnew)